In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from scipy.sparse import csr_matrix
import gzip
import itertools
import random
import sys
import math
import json
import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import anndata as ad
from dask import dataframe as dd
import os

/opt/applications/python/3.8.3/gnu/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/applications/python/3.8.3/gnu/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
mtg_rna = h5py.File('../../AD_data/SEA_AD/MTG/RNAseq/SEAAD_MTG_RNAseq_final-nuclei.2023-05-05.h5ad')
exp = ad.read_h5ad('../../AD_data/SEA_AD/MTG/RNAseq/SEAAD_MTG_RNAseq_final-nuclei.2023-05-05.h5ad')
gene_id = mtg_rna.get('var').get('gene_ids')
gene_id = pd.DataFrame(data=gene_id,columns=['gids'])
gene_id=gene_id.iloc[:,0].apply(lambda s: s.decode('utf-8'))
gene_id = dict(zip(gene_id.index,gene_id.iloc[:]))
braak = mtg_rna.get('obs').get('Braak')
braak = pd.DataFrame(data=braak)
braak_categories = mtg_rna.get('obs').get('__categories').get('Braak')
braak_categories = pd.DataFrame(data=braak_categories)
braak_categories = braak_categories.iloc[:,0].apply(lambda s: s.decode('utf-8'))
subclass = mtg_rna.get('obs').get('Subclass')
subclass = pd.DataFrame(data=subclass)

In [3]:
subclass_categories = mtg_rna.get('obs').get('__categories').get('Subclass')
subclass_categories = pd.DataFrame(data=subclass_categories)
subclass_categories = subclass_categories.iloc[:,0].apply(lambda s: s.decode('utf-8'))

In [4]:
ref_astrocytes = list(np.where((braak.iloc[:,0]==0) & (subclass.iloc[:,0]==18))[0])
br6_astrocytes = list(np.where((braak.iloc[:,0]==6) & (subclass.iloc[:,0]==18))[0])
ref_microglia = list(np.where((braak.iloc[:,0]==0) & (subclass.iloc[:,0]==23))[0])
br6_microglia = list(np.where((braak.iloc[:,0]==6) & (subclass.iloc[:,0]==23))[0])

In [5]:
counts_astrocytes = pd.DataFrame(data=exp[sorted(ref_astrocytes+br6_astrocytes)].X.toarray(),columns=gene_id.values())
counts_microglia = pd.DataFrame(data=exp[sorted(ref_microglia+br6_microglia)].X.toarray(),columns=gene_id.values())
del exp

In [6]:
counts_astrocytes

,MIR1302-2HG,FAM138A,OR4F5,AL627309.1,AL627309.3,AL627309.2,AL627309.5,AL627309.4,AP006222.2,AL732372.1,...,AC133551.1,AC136612.1,AC136616.1,AC136616.3,AC136616.2,AC141272.1,AC023491.2,AC007325.1,AC007325.4,AC007325.2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16188,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16189,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16191,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
meta_astrocytes = pd.DataFrame(data = braak.iloc[sorted(ref_astrocytes+br6_astrocytes),:])
meta_astrocytes = meta_astrocytes.reset_index(drop=True)
meta_astrocytes.columns=['condition']
meta_astrocytes.condition = np.where(meta_astrocytes.condition==0,'C','AD')
meta_microglia = pd.DataFrame(data = braak.iloc[sorted(ref_microglia+br6_microglia),:])
meta_microglia = meta_microglia.reset_index(drop=True)
meta_microglia.columns=['condition']
meta_microglia.condition = np.where(meta_microglia.condition==0,'C','AD')

In [8]:
meta_astrocytes

,condition
0,AD
1,C
2,AD
3,AD
4,AD
...,...
16188,AD
16189,AD
16190,AD
16191,AD


In [9]:
counts_astrocytes.to_csv('../../results/SEA_AD/MTG/counts_astrocytes_ref_br6.tsv',sep='\t',index=False)
counts_microglia.to_csv('../../results/SEA_AD/MTG/counts_microglia_ref_br6.tsv',sep='\t',index=False)
meta_astrocytes.to_csv('../../results/SEA_AD/MTG/meta_astrocytes_ref_br6.tsv',sep='\t',index=False)
meta_microglia.to_csv('../../results/SEA_AD/MTG/meta_microglia_ref_br6.tsv',sep='\t',index=False)

In [10]:
dds_astrocytes = DeseqDataSet(counts=counts_astrocytes.round(), metadata=meta_astrocytes, design_factors='condition')

/opt/applications/python/3.8.3/gnu/lib/python3.8/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [10]:
dds_astrocytes.deseq2()

Fitting size factors...
... done in 3.71 seconds.

Fitting dispersions...
/opt/applications/python/3.8.3/gnu/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
dds_astrocytes

In [ ]:
stat_res_astrocytes = DeseqStats(dds_astrocytes, n_cpus=8, contrast = ('Condition','AD','C'))

In [ ]:
stat_res_astrocytes.summary()

In [ ]:
result_astrocytes = stat_res_astrocytes.results_df

In [ ]:
dds_microglia = DeseqDataSet(counts=counts_microglia.round(), metadata=meta_microglia, design_factors='condition')
dds_microglia.deseq2()
dds_microglia
stat_res_microglia = DeseqStats(dds_microglia, n_cpus=8, contrast = ('Condition','AD','C'))
stat_res_microglia.summary()
result_microglia = stat_res_microglia.results_df